In [ ]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import time
import pickle
import re

In [ ]:
match_urls = []

# create a base URL
base_url = 'https://fbref.com'
schedule_url = 'https://fbref.com/en/comps/20/2016-2017/schedule/2016-2017-Bundesliga-Scores-and-Fixtures'


# use request to get URL
schedule_page = requests.get(schedule_url)


# parse URL / content of the page; need to find the url to all matches
soup = BeautifulSoup(schedule_page.content, 'html.parser')
results = soup.find(id="sched_all")
matches = results.find_all("td", class_="center")

# go trough matches and access the url and combine it with the base url for a working url to the match
for match in matches:
    links = match.find_all("a")
    for link in links:
        url = base_url + link["href"]
        match_urls.append(url)

        
with open('MatchURLs', 'wb') as fp:
        pickle.dump(match_urls, fp)        
        
        
#with open('MatchURLs', 'rb') as fp:
#        match_urls = pickle.load(fp)


In [ ]:
Home = []
Away = []
home_pos = []
away_pos = []

for match_url in match_urls:
    time.sleep(12)
    text = []
    percentages = []


    # use request to get URL
    match_page = requests.get(match_url)

    # parse URL / content of the page
    soup = BeautifulSoup(match_page.content, 'html.parser')

    results = soup.find(id="content")
    scoreboxes = results.find_all("div", class_="scorebox")
    for scorebox in scoreboxes:
        ass = scorebox.find_all("a")
        for a in ass:
            text.append(a.text)

    Home.append(text[0])
    Away.append(text[4])
    
    results = soup.find(id="team_stats")
    tables = results.find_all('table')
    for table in tables:
        trs = table.find_all('tr')
        for tr in trs:
            tds = tr.find_all('td')
            for td in tds:
                s = td.find_all('strong')
                percentages.append(str(s).replace('[<strong>', '').replace('%</strong>]', ''))

    home_pos.append(percentages[0])
    away_pos.append(percentages[1])
    


In [ ]:
possession_overview = pd.DataFrame(list(zip(Home, Away, home_pos, away_pos)),
                                           columns = ['Home', 'Away', 'Pos_Home', 'Pos_Away'])
possession_overview

In [ ]:
possession_overview = possession_overview[:306]
possession_overview.to_csv('Output/PossessionOverview.csv')

In [ ]:
# match names

game_overview2= pd.read_csv('Output/GameOverview2.csv')
game_overview2.Home.unique()

In [ ]:
possession_overview = pd.read_csv('Output/PossessionOverview.csv')
possession_overview = possession_overview.iloc[:, 1:]
possession_overview.Home.unique()

In [ ]:
possession_overview.replace(possession_overview.Home.unique(), ['FC Bayern München', 'Borussia Dortmund', 'Hamburger SV', 
                                                               'Eintracht Frankfurt', '1. FC Köln', 'FC Augsburg', 
                                                               'Borussia Mönchengladbach', 'Hertha BSC', 'TSG 1899 Hoffenheim',
                                                               'FC Schalke 04', 'Bayer 04 Leverkusen', 'Sport-Club Freiburg',
                                                               'VfL Wolfsburg', 'FC Ingolstadt 04', 'SV Darmstadt 98',
                                                               'RB Leipzig', 'SV Werder Bremen', '1. FSV Mainz 05'],
                            inplace= True)
possession_overview

In [ ]:
new_overview = pd.merge(game_overview2, possession_overview, on = ['Home', 'Away'], how = 'inner')
new_overview = new_overview.iloc[:,1:]

new_overview.to_csv('Output/GameOverview3.csv')
new_overview